## 문서 벡터를 이용한 추천 시스템(Recommendation System using Document Embedding)

### 1. 데이터 로드

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 규원님이 전처리 완료하신 최종 데이터 (+라벨까지 붙은)
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
len(df)

74779

In [4]:
user_setence = 'I am sitting on the beach with a cool breeze I am surrounded by coconut palm water and I sip a refreshing grapefruit sparkling drink'

In [5]:
user_setence = user_setence.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
user_setence = ' '.join([w for w in user_setence.split() if len(w)>3])
# 전체 단어에 대한 소문자 변환
user_setence = user_setence.lower()

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# NLTK로부터 불용어 로드
stop_words = stopwords.words('english') 
tokenized_doc = user_setence.split() # 토큰화
tokenized_doc = [item for item in tokenized_doc if item not in stop_words] # 불용어 제거

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
n = WordNetLemmatizer()
tokenized_doc = [n.lemmatize(item) for item in tokenized_doc] # 표제어 추출
print(tokenized_doc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['sitting', 'beach', 'cool', 'breeze', 'surrounded', 'coconut', 'palm', 'water', 'refreshing', 'grapefruit', 'sparkling', 'drink']


분류된 라벨이 0이라고 가정할 경우

In [7]:
df2 = df[df['label']==0]

In [8]:
sent_text = df2['lemmatizated']

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
result.append(tokenized_doc)

In [11]:
for line in result[:3]:
  print(line)

['really', 'sorry', 'say', 'smell', 'toilet', 'freshener']
['really', 'really', 'sorry', 'say', 'coworker', 'wore', 'long', 'time', 'think', 'hamster', 'cage', 'cubicle']
['perfume', 'make', 'sad', 'first', 'minute', 'spray', 'soooo', 'lovely', 'open', 'lovely', 'citrus', 'note', 'feel', 'bright', 'fresh', 'perfect', 'fragrance', 'casual', 'summer', 'day', 'unfortunately', 'worst', 'stay', 'power', 'perfume', 've', 'try', 'seriously', 'within', 'minute', 've', 'smell', 'fart', 'wind', 'lasted', 'long', 'shame', 'beautiful', 'scent', 'would', 'great', 'every', 'day', 'use', 'casual', 'still', 'beautiful', 'would', 'absolutely', 'buy', 'price', 'take', 'account', 'awful', 'stay', 'power', 'unfortunately', 'n', 't', 'justify', 'price', 'usually', 'fragrances', 'last', 'ish', 'hour', 'two', 'spray', 'use', 'spray', 'minute', 'know', 'edt', 'edts', 'last', 'lot', 'longer']


In [33]:
len(result)

20103

### 2. 사전 훈련된 워드 임베딩 로드하여 단어 벡터 평균 계산

In [13]:
# 모델 로드
from gensim.models import Word2Vec, KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_10window') # 모델 로드

In [38]:
# 단어 벡터 평균 구하기
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line:
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]
        
        if doc2vec is None:
            doc2vec = np.empty(100,)
            doc2vec[:] = 0
            document_embedding_list.append(doc2vec)
        else:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [39]:
document_embedding_list = vectors(result)
print('분류된 라벨의 문서 벡터의 수 :',len(document_embedding_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


분류된 라벨의 문서 벡터의 수 : 20103


### 3. 문서 간 유사도 계산

향수 데이터에서는 전체 문서간의 코사인 유사도 매트릭스가 아닌 같은 라벨 내에서 사용자 입력문장과의 유사도 매트릭스를 구해야 함

In [40]:
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (20103, 20103)


In [61]:
def recommendations(df, cosine_similarities):
    perfumes = df[['name', 'review']]

    # 전체 cosine유사도 행렬에서 사용자 입력 문장과 가장 유사한 순으로 리뷰 정렬
    sim_scores = list(enumerate(cosine_similarities[-1]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:100]

    # 가장 유사한 리뷰 10개의 인덱스
    per_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = df.iloc[per_indices].reset_index(drop=True)


    # 데이터프레임으로부터 순차적으로 출력
    recommend_perfume = []
    for index, row in recommend.iterrows():
      if len(recommend_perfume)==3:
        break
      if row['name'] in recommend_perfume:
        continue
      else:
        recommend_perfume.append(row['name'])
      print('Top {}'.format(len(recommend_perfume)))
      print('향수 명: ' ,row['name'])
      print('유사도: ',sim_scores[index][1])
      print('리뷰: ', row['review'])
      print()
      print()

In [62]:
recommendations(df2,cosine_similarities)

Top 1
향수 명:  Light Blue Dolce&Gabbana for women
유사도:  0.9269124360695515
리뷰:  This D&G is reminiscent of a golden beach in summer. Lemon and lime decorating my pina colada glass and I can smell their charm as I dip my lips into that glass of tropical heaven; the sun melting fruit juice onto my mocha-tan face. 
There's a citrus zing floating mysteriously through the air as I lay and stare contently at the sky that is a blue of sorts. Light blue though, is the breeze beneath it, convincing you all to buy a bottle of such sweetness.


Top 2
향수 명:  Un Jardin Sur Le Nil Hermès for women and men
유사도:  0.9180963901992103
리뷰:  The boat along the green waters of the Nile delta, or any delta. The light splash of an aquatic creature. The sun reflected into the crystaline waves and bloom of the nufars. All the luxury of the water and its lush offspring in a scent that retains the so called "aquatic notes" in an authentic and unfading way, like none other perfume can do. 
Un Jardin sur le Nile is t